In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import numpy as np
import requests

In [2]:
link_jour_ferie = 'https://www.data.gouv.fr/fr/datasets/r/6637991e-c4d8-4cd6-854e-ce33c5ab49d5' # risque obsolescence
link_vacance_scolaire = 'https://www.data.gouv.fr/fr/datasets/r/9957d723-346e-4317-8cb3-293c94e19b2d' # risque obsolescence
debut_calendrier = '2017-09-01' # premiere vacance dispo dans le data set de data gouv en octobre 2017
fin_calendrier = '2026-06-28' # derniere vacance dispo  dans le data set de data gouv en avril 2026

In [3]:
df_jour_ferie = pd.read_csv(link_jour_ferie)
df_jour_ferie['date'] = pd.to_datetime(df_jour_ferie['date'])
df_jour_ferie

,date,annee,zone,nom_jour_ferie
0,2003-01-01,2003,Métropole,1er janvier
1,2003-04-21,2003,Métropole,Lundi de Pâques
2,2003-05-01,2003,Métropole,1er mai
3,2003-05-08,2003,Métropole,8 mai
4,2003-05-29,2003,Métropole,Ascension
...,...,...,...,...
281,2028-07-14,2028,Métropole,14 juillet
282,2028-08-15,2028,Métropole,Assomption
283,2028-11-01,2028,Métropole,Toussaint
284,2028-11-11,2028,Métropole,11 novembre


In [4]:
df_vacance_scolaire = pd.read_csv(link_vacance_scolaire,sep=';')
df_vacance_scolaire = df_vacance_scolaire[df_vacance_scolaire['zones'].isin(['Zone A','Zone B','Zone C'])]
df_vacance_scolaire=df_vacance_scolaire.drop_duplicates(subset = ['zones','description','annee_scolaire'])
df_vacance_scolaire['start_date'] = pd.to_datetime(df_vacance_scolaire['start_date'].str[0:10])
df_vacance_scolaire['end_date'] = pd.to_datetime(df_vacance_scolaire['end_date'].str[0:10])
df_vacance_scolaire

,description,population,start_date,end_date,location,zones,annee_scolaire
11,Vacances de la Toussaint,-,2017-10-21,2017-11-06,Caen,Zone B,2017-2018
16,Vacances de Noël,-,2017-12-23,2018-01-08,Clermont-Ferrand,Zone A,2017-2018
18,Vacances de Noël,-,2017-12-23,2018-01-08,Orléans-Tours,Zone B,2017-2018
21,Vacances de Noël,-,2017-12-23,2018-01-08,Créteil,Zone C,2017-2018
27,Vacances d'Hiver,-,2018-02-17,2018-03-05,Montpellier,Zone C,2017-2018
...,...,...,...,...,...,...,...
910,Vacances de Noël,-,2019-12-21,2020-01-06,Créteil,Zone C,2019-2020
934,Vacances d'Été,-,2020-07-04,2020-09-01,Paris,Zone C,2019-2020
944,Vacances de la Toussaint,-,2020-10-17,2020-11-02,Créteil,Zone C,2020-2021
1069,Vacances de Noël,-,2023-12-23,2024-01-08,Versailles,Zone C,2023-2024


In [5]:
df_calendar = pd.DataFrame( index = pd.date_range(start=debut_calendrier, end=fin_calendrier))
df_calendar['is_ferie']=0
df_calendar.loc[df_calendar.index.isin(df_jour_ferie['date']),'is_ferie']=1
df_calendar = df_calendar.merge(df_jour_ferie, how='left', left_on = df_calendar.index, right_on ='date').set_index('date').drop(columns=['annee','zone'])

In [6]:
df_calendar['Zone A']=0
df_calendar['Zone B']=0
df_calendar['Zone C']=0

df_calendar['Zone A vacance']=np.nan
df_calendar['Zone B vacance']=np.nan
df_calendar['Zone C vacance']=np.nan

In [7]:
for index,row in df_vacance_scolaire.iterrows():
    vacance_name = row['description']
    date_debut = row['start_date']
    date_fin = row['end_date']
    zone_vac = row['zones']
    df_calendar.loc[slice(date_debut,date_fin-timedelta(minutes=1)),zone_vac]=1
    df_calendar.loc[slice(date_debut,date_fin-timedelta(minutes=1)),zone_vac+' vacance']=vacance_name

In [8]:
df_calendar.to_csv('/data/prediction_frequentation/calendar_complete_from_data_gouv.csv')